# 2-3,动态计算图

本节我们将介绍 Pytorch的动态计算图。

包括： 

* 动态计算图简介

* 计算图中的Function

* 计算图和反向传播

* 叶子节点和非叶子节点

* 计算图在TensorBoard中的可视化



### 一，动态计算图简介

![](./data/torch动态图.gif)

Pytorch的计算图由节点和边组成，节点表示张量或者Function，边表示张量和Function之间的依赖关系。

Pytorch中的计算图是动态图。这里的动态主要有两重含义。

第一层含义是：计算图的正向传播是立即执行的。无需等待完整的计算图创建完毕，每条语句都会在计算图中动态添加节点和边，并立即执行正向传播得到计算结果。

第二层含义是：计算图在反向传播后立即销毁。下次调用需要重新构建计算图。如果在程序中使用了backward方法执行了反向传播，或者利用torch.autograd.grad方法计算了梯度，那么创建的计算图会被立即销毁，释放存储空间，下次调用需要重新创建。


**1，计算图的正向传播是立即执行的。**

In [ ]:
# 导入PyTorch库
import torch

# 创建一个可训练的张量w，包含一个2维的行向量，requires_grad=True表示会跟踪梯度
w = torch.tensor([[3.0, 1.0]], requires_grad=True)

# 创建一个可训练的张量b，包含一个标量，requires_grad=True表示会跟踪梯度
b = torch.tensor([[3.0]], requires_grad=True)

# 创建一个随机张量X，大小为10x2，用于输入数据
X = torch.randn(10, 2)

# 创建一个随机张量Y，大小为10x1，用于目标数据
Y = torch.randn(10, 1)

# 计算预测值Y_hat，这里使用矩阵乘法（@）和转置（t()）来计算
Y_hat = X @ w.t() + b
# Y_hat定义后，正向传播被立即执行，与其后面的loss创建语句无关

# 计算损失，这里使用均方误差损失（MSE）
loss = torch.mean(torch.pow(Y_hat - Y, 2))

# 打印损失值
print(loss.data)

# 打印预测值Y_hat
print(Y_hat.data)


**2，计算图在反向传播后立即销毁。**

In [ ]:
# 导入PyTorch库
import torch

# 创建一个可训练的张量w，包含一个2维的行向量，requires_grad=True表示会跟踪梯度
w = torch.tensor([[3.0, 1.0]], requires_grad=True)

# 创建一个可训练的张量b，包含一个标量，requires_grad=True表示会跟踪梯度
b = torch.tensor([[3.0]], requires_grad=True)

# 创建一个随机张量X，大小为10x2，用于输入数据
X = torch.randn(10, 2)

# 创建一个随机张量Y，大小为10x1，用于目标数据
Y = torch.randn(10, 1)

# 计算预测值Y_hat，这里使用矩阵乘法（@）和转置（t()）来计算
Y_hat = X @ w.t() + b
# Y_hat定义后，正向传播被立即执行，与其后面的loss创建语句无关

# 计算损失，这里使用均方误差损失（MSE）
loss = torch.mean(torch.pow(Y_hat - Y, 2))

# 使用backward()方法进行反向传播计算梯度
loss.backward()
# 如果需要保留计算图，可以使用retain_graph=True参数 loss.backward(retain_graph = True) ，但通常情况下不需要

# 如果再次执行反向传播将报错，因为默认情况下计算图在反向传播后会被销毁
# 如果需要多次反向传播，需要在第一次反向传播时使用retain_graph=True


### 二，计算图中的Function

计算图中的 张量我们已经比较熟悉了, 计算图中的另外一种节点是Function, 实际上就是 Pytorch中各种对张量操作的函数。

这些Function和我们Python中的函数有一个较大的区别，那就是它同时包括正向计算逻辑和反向传播的逻辑。

我们可以通过继承torch.autograd.Function来创建这种支持反向传播的Function


In [ ]:
import torch


# 自定义ReLU激活函数类，继承自torch.autograd.Function
class MyReLU(torch.autograd.Function):

    # 正向传播逻辑，ctx用于存储一些值以便反向传播使用
    @staticmethod
    def forward(ctx, input):
        # 保存输入张量，以备反向传播使用
        ctx.save_for_backward(input)
        # ReLU激活函数的正向传播逻辑：将小于0的值设为0，大于等于0的值保持不变
        return input.clamp(min=0)

    # 反向传播逻辑
    @staticmethod
    def backward(ctx, grad_output):
        # 从ctx中取出保存的输入张量
        input, = ctx.saved_tensors
        # 克隆梯度张量，用于计算梯度
        grad_input = grad_output.clone()
        # 根据ReLU激活函数的导数，将小于0的梯度设为0
        grad_input[input < 0] = 0
        return grad_input


In [ ]:
import torch

# 创建可训练的权重张量w，包含一个2维的行向量，requires_grad=True表示会跟踪梯度
w = torch.tensor([[3.0, 1.0]], requires_grad=True)

# 创建可训练的偏置张量b，包含一个标量，requires_grad=True表示会跟踪梯度
b = torch.tensor([[3.0]], requires_grad=True)

# 创建输入张量X，大小为2x2
X = torch.tensor([[-1.0, -1.0], [1.0, 1.0]])

# 创建目标张量Y，大小为1x2
Y = torch.tensor([[2.0, 3.0]])

# 通过MyReLU.apply将自定义的ReLU函数应用于X @ w.t() + b的结果
# 这里relu现在也可以具有正向传播和反向传播功能
relu = MyReLU.apply
Y_hat = relu(X @ w.t() + b)

# 计算损失，这里使用均方误差损失（MSE）
loss = torch.mean(torch.pow(Y_hat - Y, 2))

# 使用backward()方法进行反向传播计算梯度
loss.backward()

# 打印权重w的梯度
print(w.grad)

# 打印偏置b的梯度
print(b.grad)


In [ ]:
# Y_hat的梯度函数即是我们自己所定义的 MyReLU.backward

print(Y_hat.grad_fn)

### 三，计算图与反向传播

了解了Function的功能，我们可以简单地理解一下反向传播的原理和过程。理解该部分原理需要一些高等数学中求导链式法则的基础知识。


In [ ]:
import torch

# 创建一个可训练的标量张量x，requires_grad=True表示会跟踪梯度
x = torch.tensor(3.0, requires_grad=True)

# 定义两个新的张量y1和y2，它们是基于x的计算得到的
y1 = x + 1
y2 = 2 * x

# 定义损失函数，这里是(y1 - y2)^2
loss = (y1 - y2) ** 2

# 使用backward()方法进行反向传播计算梯度
loss.backward()

# 此时x.grad包含了损失相对于x的梯度值
print(x.grad)


loss.backward()语句调用后，依次发生以下计算过程。

1，loss自己的grad梯度赋值为1，即对自身的梯度为1。

2，loss根据其自身梯度以及关联的backward方法，计算出其对应的自变量即y1和y2的梯度，将该值赋值到y1.grad和y2.grad。

3，y2和y1根据其自身梯度以及关联的backward方法, 分别计算出其对应的自变量x的梯度，x.grad将其收到的多个梯度值累加。

（注意，1,2,3步骤的求梯度顺序和对多个梯度值的累加规则恰好是求导链式法则的程序表述）

正因为求导链式法则衍生的梯度累加规则，张量的grad梯度不会自动清零，在需要的时候需要手动置零。


### 四，叶子节点和非叶子节点

执行下面代码，我们会发现 loss.grad并不是我们期望的1,而是 None。

类似地 y1.grad 以及 y2.grad也是 None.

这是为什么呢？这是由于它们不是叶子节点张量。

**在反向传播过程中，只有 is_leaf=True 的叶子节点，需要求导的张量的导数结果才会被最后保留下来。**

那么什么是叶子节点张量呢？叶子节点张量需要满足两个条件。

- 叶子节点张量是由用户直接创建的张量，而非由某个Function通过计算得到的张量。

- 叶子节点张量的 requires_grad属性必须为True.

Pytorch设计这样的规则主要是为了节约内存或者显存空间，因为几乎所有的时候，用户只会关心他自己直接创建的张量的梯度。

所有依赖于叶子节点张量的张量, 其requires_grad 属性必定是True的，但其梯度值只在计算过程中被用到，不会最终存储到grad属性中。

如果需要保留中间计算结果的梯度到grad属性中，可以使用 retain_grad方法。
如果仅仅是为了调试代码查看梯度值，可以利用register_hook打印日志。


In [ ]:
import torch  # 导入PyTorch库

# 创建一个张量x，值为3.0，并启用梯度计算
x = torch.tensor(3.0, requires_grad=True)

# 计算y1，它是x + 1的结果
y1 = x + 1

# 计算y2，它是2 * x的结果
y2 = 2 * x

# 计算损失，这里是(y1 - y2)的平方
loss = (y1 - y2) ** 2

# 执行自动微分，计算梯度
loss.backward()

# 打印损失（loss）的梯度
print("loss.grad:", loss.grad)

# 打印y1的梯度
print("y1.grad:", y1.grad)

# 打印y2的梯度
print("y2.grad:", y2.grad)

# 打印x的梯度，这是我们最关心的，它表示损失对x的偏导数
print(x.grad)


In [ ]:
# 检查x是否是叶子节点
print(x.is_leaf)  # 输出 True，因为x是由用户创建的，是计算图的起点

# 检查y1是否是叶子节点
print(y1.is_leaf)  # 输出 False，因为y1是通过对x进行操作而自动创建的，不是叶子节点

# 检查y2是否是叶子节点
print(y2.is_leaf)  # 输出 False，因为y2也是通过对x进行操作而自动创建的，不是叶子节点

# 检查loss是否是叶子节点
print(loss.is_leaf)  # 输出 False，因为loss是通过对y1和y2进行操作而自动创建的，不是叶子节点


利用retain_grad可以保留非叶子节点的梯度值，利用register_hook可以查看非叶子节点的梯度值。

In [ ]:
import torch  # 导入PyTorch库

# 正向传播
x = torch.tensor(3.0, requires_grad=True)  # 创建一个张量x，值为3.0，并启用梯度计算
y1 = x + 1  # 计算y1，它是x + 1的结果
y2 = 2 * x  # 计算y2，它是2 * x的结果
loss = (y1 - y2) ** 2  # 计算损失，这里是(y1 - y2)的平方

# 控制非叶子节点的梯度显示
y1.register_hook(lambda grad: print('y1 grad: ', grad))  # 使用hook函数打印y1的梯度
y2.register_hook(lambda grad: print('y2 grad: ', grad))  # 使用hook函数打印y2的梯度
loss.retain_grad()  # 保留损失的梯度以供显示

# 反向传播
loss.backward()  # 执行反向传播计算梯度

# 打印损失（loss）的梯度
print("loss.grad:", loss.grad)  # 输出损失的梯度

# 打印x的梯度，这是我们最关心的，它表示损失对x的偏导数
print("x.grad:", x.grad)  # 输出x的梯度


### 五，计算图在TensorBoard中的可视化

可以利用 torch.utils.tensorboard 将计算图导出到 TensorBoard进行可视化。

In [ ]:
from torch import nn  # 导入PyTorch的nn模块


# 定义一个神经网络类Net，继承自nn.Module
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()  # 调用父类构造函数
        self.w = nn.Parameter(torch.randn(2, 1))  # 定义一个可学习的参数w，大小为(2, 1)，并用随机初始化
        self.b = nn.Parameter(torch.zeros(1, 1))  # 定义一个可学习的参数b，大小为(1, 1)，并用零初始化

    def forward(self, x):
        y = x @ self.w + self.b  # 前向传播计算，将输入x与参数w相乘并加上参数b
        return y


# 创建一个Net实例，这个实例可以用于进行前向传播和后向传播
net = Net()


In [ ]:
from torch.utils.tensorboard import SummaryWriter

writer = SummaryWriter('./data/tensorboard')
writer.add_graph(net, input_to_model=torch.rand(10, 2))
writer.close()
%load_ext tensorboard
#%tensorboard --logdir ./data/tensorboard
from tensorboard import notebook

notebook.list()
#在tensorboard中查看模型
notebook.start("--logdir ./data/tensorboard")